# ETL Pipeline for Pterodactyl Minecraft Servers Analysis

## Index

- Install requierements
- Import libraries and setup key variables
- Setup folders and directories
- Define functions
- Folder creation if not exist
- Get Pterodactyl Application information
- Upload csv table files into Postgres
- Extract logs from each active Minecraft Server
- Transformation from logs data into information
- Load processed data into Data Warehouse (Postgres)

## Install requierements

In [ ]:
!pip install -r requirements.txt

## Import libraries and setup key variables
Remember to add you own credentials in the .env file for them to be loaded here

In [ ]:
import urllib.request, psycopg2, pyarrow, json, csv, gzip, re, os
from sqlalchemy import create_engine, text
from pydactyl import PterodactylClient
from dotenv import load_dotenv
import pandas as pd
import datetime

# Load .env file credentials
load_dotenv()

# Database connection
host = os.getenv('POSTGRES_HOST')
port = os.getenv('POSTGRES_PORT')
dbname = os.getenv('POSTGRES_DBNAME')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
connection = f'postgresql://{user}:{password}@{host}:{port}/{dbname}'

# Pterodactyl connection
pterodactyl_url = os.getenv('PTERODACTYL_URL')
application_api_key = os.getenv('PTERODACTYL_APPLICATION_API_KEY')
client_api_key = os.getenv('PTERODACTYL_CLIENT_API_KEY')

# Connecto to Pterodactyl Application API
api_app = PterodactylClient(pterodactyl_url, application_api_key, debug=False)
# Connecto to Pterodactyl Client API
api_cli = PterodactylClient(pterodactyl_url, client_api_key, debug=False)

## Setup folders and directories

In [ ]:
pwd = os.getcwd() #os.path.dirname(os.path.realpath(__file__)) this is used for .py files
server_app_folder = os.path.join(pwd, 'server_app_data')
raw_logs_folder = os.path.join(pwd, 'raw_logs')
#output_folder = ''

## Define functions

In [ ]:
# Create new folder if not exists
def mkdir(folder_dir):
    if not os.path.exists(folder_dir):
        os.makedirs(os.path.join(pwd, folder_dir))

# Export data into a .csv file
def save_to_csv(data, filename):
    with open(os.path.join(pwd, server_app_folder, filename), 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = data[0][0].keys()  # Assuming the data is not empty
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for record in data:
            for item in record:
                writer.writerow(item)

# Sort a list of logs names
def sort_list_logs(logs):
    def logs_modifications(log):
        # remove the '-' from the log
        date_part, number_part = log.split('-')[:3], log.split('-')[3]
        # Join the parts of date to transform
        date_log = '-'.join(date_part)
        # remove the '.log' extension and pass the number to int
        number_log = int(number_part.split('.')[0])
        # return the date and number for sorting
        return date_log, number_log

    # use the logs_modifications function to sort the logs by date and number
    sorted_logs = sorted(logs, key=logs_modifications)
    return sorted_logs

## Folder creation if not exist

In [ ]:
mkdir(server_app_folder)
mkdir(raw_logs_folder)

## Get Pterodactyl Application information
About: locations, nodes, nests, eggs, servers, clients

In [ ]:
# Extracting data from Pterodactyl App
list_of_clients = api_app.user.list_users()
all_clients = [[client['attributes']for client in clients]for clients in list_of_clients]

list_of_locations = api_app.locations.list_locations()
all_locations = [[location['attributes']for location in locations]for locations in list_of_locations]

list_of_nodes = api_app.nodes.list_nodes()
all_nodes = [[node['attributes']for node in nodes]for nodes in list_of_nodes]

list_of_nests_and_eggs = api_app.nests.list_nests(includes=['eggs'])
all_nests = [[nest['attributes'] for nest in nests] for nests in list_of_nests_and_eggs]
all_eggs = [[eggs['attributes'] for eggs in nests['attributes']['relationships']['eggs']['data']] for nests in list_of_nests_and_eggs]

list_of_servers_and_clients = api_app.servers.list_servers(includes=['subusers'])
all_servers = [[server['attributes']for server in servers] for servers in list_of_servers_and_clients]
all_client_server = [[client_server['attributes'] for client_server in servers['attributes']['relationships']['subusers']['data']] for servers in list_of_servers_and_clients]

# Get the current timestamp with timezone information (UTC)
last_update = datetime.datetime.now(datetime.timezone.utc)

# Cleaning and filtering columns
df_clients = pd.DataFrame(all_clients[0])
df_clients = df_clients[['id', 'uuid', 'username', 'email', 'first_name', 'last_name', 'root_admin', '2fa', 'created_at', 'updated_at']].rename(columns={'username': 'client_name', 'root_admin': 'admin'})

df_locations = pd.DataFrame(all_locations[0])
df_locations = df_locations[['id', 'short', 'long', 'created_at', 'updated_at']]

df_nodes = pd.DataFrame(all_nodes[0])
df_nodes['allocated_memory'] = df_nodes['allocated_resources'].apply(lambda x: x.get('memory', None))
df_nodes['allocated_disk'] = df_nodes['allocated_resources'].apply(lambda x: x.get('disk', None))
df_nodes = df_nodes[['id', 'uuid', 'public', 'name', 'description', 'location_id', 'fqdn', 'scheme', 'behind_proxy', 'maintenance_mode', 'memory', 'disk', 'allocated_memory', 'allocated_disk', 'upload_size', 'daemon_listen', 'daemon_sftp', 'daemon_base','created_at', 'updated_at']].rename(columns={'': '', '': ''})

df_nests = pd.DataFrame(all_nests[0])
df_nests = df_nests[['id', 'uuid', 'name', 'description', 'author', 'created_at', 'updated_at']]

df_eggs = pd.DataFrame(all_eggs[0])
df_eggs = df_eggs[['id', 'uuid', 'name', 'description', 'nest', 'author', 'created_at', 'updated_at']].rename(columns={'nest': 'nest_id'})

df_servers = pd.DataFrame(all_servers[0])
df_servers['limit_memory'] = df_servers['limits'].apply(lambda x: x.get('memory', None))
df_servers['limit_disk'] = df_servers['limits'].apply(lambda x: x.get('disk', None))
df_servers['limit_io'] = df_servers['limits'].apply(lambda x: x.get('io', None))
df_servers['limit_cpu'] = df_servers['limits'].apply(lambda x: x.get('cpu', None))
df_servers['limit_oom_disable'] = df_servers['limits'].apply(lambda x: x.get('oom_disable', None))
df_servers['limit_database'] = df_servers['feature_limits'].apply(lambda x: x.get('database', None))
df_servers['limit_allocation'] = df_servers['feature_limits'].apply(lambda x: x.get('allocation', None))
df_servers['limit_backup'] = df_servers['feature_limits'].apply(lambda x: x.get('backup', None))
df_servers = df_servers[['id', 'uuid', 'identifier', 'name', 'description', 'limit_memory', 'limit_disk', 'limit_io', 'limit_cpu', 'limit_oom_disable', 'limit_database', 'limit_allocation', 'limit_backup', 'user', 'node', 'allocation', 'nest', 'egg','created_at', 'updated_at']].rename(columns={'user': 'client_id', 'node': 'node_id', 'allocation': 'allocation_id', 'nest': 'nest_id', 'egg': 'egg_id'})

flattened_client_server = [item for sublist in all_client_server for item in sublist]
df_clients_server = pd.DataFrame(flattened_client_server)[['id', 'user_id', 'server_id', 'created_at', 'updated_at']].rename(columns={'user_id': 'client_id'})
df_clients_server

# Exporting data into .csv files
df_clients.to_csv(os.path.join(server_app_folder, 'clients.csv'), index=False)
df_locations.to_csv(os.path.join(server_app_folder,'locations.csv'), index=False)
df_nodes.to_csv(os.path.join(server_app_folder,'nodes.csv'), index=False)
df_nests.to_csv(os.path.join(server_app_folder,'nests.csv'), index=False)
df_eggs.to_csv(os.path.join(server_app_folder,'eggs.csv'), index=False)
df_servers.to_csv(os.path.join(server_app_folder,'servers.csv'), index=False)
df_clients_server.to_csv(os.path.join(server_app_folder,'clients_server.csv'), index=False)

## Upload csv table files into Postgres

In [ ]:
engine = create_engine(connection)

for file_table in os.listdir(server_app_folder):

    # Setup database variables
    ID = 'id'
    TABLE = file_table.split('.')[0]
    TABLE_UPDATE = TABLE + '_update'
    IS_ACTIVE_TABLE = 'is_active_table'
    SCHEMA = 'pterodactyl'
    
    # Reading of the file_table
    df = pd.read_csv(os.path.join(server_app_folder, file_table))

    # Start connection with database
    with engine.connect() as conn:
        # Start a new transaction
        trans = conn.begin()

        try:
            # Load ID from database
            result = conn.execute(text(f'SELECT "{ID}" FROM {SCHEMA}.{TABLE}'))
            db = pd.DataFrame(result.fetchall(), columns=result.keys())

            # Compare ID
            sameID = db[ID].isin(df[ID])

            toUpdate = df[df[ID].isin(db[ID][sameID])]
            toIngest = df[~df[ID].isin(db[ID][sameID])]
            toDelete = db[~db[ID].isin(df[ID])]

            # Insert the DataFrame into a table
            toIngest.to_sql(TABLE, conn, schema=SCHEMA, if_exists='append', index=False)

            # Insert the updatable DataFrame into the TABLE_UPDATE table
            toUpdate.to_sql(TABLE_UPDATE, conn, schema=SCHEMA, if_exists='append', index=False)

            # Define and execute the following queries
            conn.execute(text(f'DELETE FROM {SCHEMA}.{TABLE} WHERE "{ID}" IN (SELECT "{ID}" FROM {SCHEMA}.{TABLE_UPDATE});'))
            conn.execute(text(f'INSERT INTO {SCHEMA}.{TABLE} SELECT * FROM {SCHEMA}.{TABLE_UPDATE};'))
            conn.execute(text(f'TRUNCATE TABLE {SCHEMA}.{TABLE_UPDATE};'))

            # Update column "is_active" from tables when data is deleted from Pterodactyl App
            toDelete.to_sql(IS_ACTIVE_TABLE, conn, schema=SCHEMA, if_exists='append', index=False)
            conn.execute(text(f'UPDATE {SCHEMA}.{TABLE} SET is_active = false WHERE "{ID}" IN (SELECT * FROM {SCHEMA}.{IS_ACTIVE_TABLE});'))
            conn.execute(text(f'TRUNCATE TABLE {SCHEMA}.{IS_ACTIVE_TABLE};'))

            # Commit the transaction
            trans.commit()

        except Exception as e:
            # Rollback the transaction on exception
            print('!!! [ERROR IN DATABASE QUERIES] !!!')
            trans.rollback()
            print('Transaction has been rolled back')
            print(f'Error occurred during transaction:\n{e}')
            raise

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()

    try:
        # Update date from the last_update table based on max date on the file
        old_last_update = conn.execute(text(f'SELECT date FROM {SCHEMA}.last_update')).fetchall()[0][0]
        new_last_update = last_update
        if new_last_update > old_last_update:
            conn.execute(text(f"UPDATE {SCHEMA}.last_update SET date = '{new_last_update}';"))

        # Commit the transaction
        trans.commit()

    except Exception as e:
        # Rollback the transaction on exception
        print('!!! [ERROR IN DATABASE QUERIES] !!!')
        trans.rollback()
        print('Transaction has been rolled back')
        print(f'Error occurred during transaction:\n{e}')
        raise

## Extract logs from each active Minecraft Server

In [ ]:
log_pattern = r'^\d{4}-\d{2}-\d{2}-\d.*' # yyyy-mm-dd-n*
extension_file_compressed = '.log.gz'
extension_file_uncompessed = '.log'
output_logs_folder = os.path.join(pwd, 'processed_logs')
SCHEMA = 'pterodactyl'

eggs_ready = ['Vanilla Minecraft', 'Forge Minecraft', 'Paper'] # Vanilla Bedrock is still not ready to be processed

# Create folder output_logs_folder
mkdir(output_logs_folder)

# Get server information
engine = create_engine(connection)
with engine.connect() as conn:
    list_servers = conn.execute(text(f'SELECT servers.identifier, eggs.name FROM {SCHEMA}.servers JOIN {SCHEMA}.eggs ON eggs.id = servers.egg_id WHERE servers.is_active = true and servers.nest_id = 1'))

for server_info in list_servers:

    if server_info[1] in eggs_ready:
        # Create a folder
        folder_name = server_info[0]
        folder_server_dir = os.path.join(raw_logs_folder, folder_name)
        mkdir(folder_server_dir)

        # Create a sub-folder called last_log inside of the folder of the server
        last_log_dir = os.path.join(folder_server_dir, 'last_log')
        mkdir(last_log_dir)

        # Try to get userchache.json file from server
        try:
            # Download users in cache
            users_cache = api_cli.client.servers.files.get_file_contents(server_info[0], 'usercache.json')
            user_names = [user['name'] for user in users_cache]
        except:
            user_names = [None]

        # Add new users only
        with open(os.path.join(raw_logs_folder, folder_name, 'users.csv'), 'a+', newline='') as csvfile:
            csvfile.seek(0)
            reader = csv.DictReader(csvfile)
            existing_names = [row['name'] for row in reader]
            new_names = [name for name in user_names if name not in existing_names]

            writer = csv.DictWriter(csvfile, fieldnames=['name'])
            if not existing_names:
                writer.writeheader()

            for name in new_names:
                writer.writerow({'name': name})

        # Download new log files
        # Select the last log name in folder last_log
        if os.listdir(last_log_dir):
            last_log_name = os.listdir(last_log_dir)[0]
        else:
            last_log_name = None
        # Get a sorted list of the logs inside the server
        log_files = api_cli.client.servers.files.list_files(server_info[0], '/logs/')
        list_logs = [file['attributes']['name'] for file in log_files['data'] if re.match(log_pattern, file['attributes']['name'])]
        sorted_list_logs = sort_list_logs(list_logs)
        # Select only a list of downloadable_logs which are new; after the last_log_name
        try:
            index_last_log = sorted_list_logs.index(last_log_name + '.gz')
        except:
            index_last_log = -1
        downloadable_logs = sorted_list_logs[index_last_log + 1:]
        # Download all logs in the list downloadable_logs
        list_download = [api_cli.client.servers.files.download_file(server_info[0], f'/logs/{log}') for log in downloadable_logs]
        if list_download:
            [urllib.request.urlretrieve(list_download[i], os.path.join(folder_server_dir, list_logs[i])) for i in range(len(list_download))]
        print(f'Files downloaded: {len(list_download)}')

        # Uncompressing files
        for filename in os.listdir(folder_server_dir):
            if filename.endswith(extension_file_compressed):
                compressed_file_path = os.path.join(folder_server_dir,filename)
                decompressed_file_path = os.path.splitext(compressed_file_path)[0] # Remove the .gz extension

                # Uncompress the file
                with gzip.open(compressed_file_path, 'rb') as compressed_file:
                    with open(decompressed_file_path, 'wb') as decompressed_file:
                        decompressed_file.write(compressed_file.read())

                # Delete the compressed file
                os.remove(compressed_file_path)

        # If there are .log files in folder_server_dir
        if [file for file in os.listdir(folder_server_dir) if re.match(log_pattern, file)]:
            # Delete last log file from folder last_log_dir
            if os.listdir(last_log_dir):
                os.remove(os.path.join(last_log_dir, os.listdir(last_log_dir)[0]))

            # Create a empty file called as the last_log in folder last_log
            list_local_logs = sort_list_logs([file for file in os.listdir(folder_server_dir) if re.match(log_pattern, file)])
            with open(os.path.join(last_log_dir, list_local_logs[-1]), 'w'):
                pass

## Transformation from logs data to information

In [ ]:
df_all_logs = pd.DataFrame(columns=['server_id', 'date', 'time', 'information', 'user', 'activity'])

for folder in os.listdir(raw_logs_folder): 
    folder_server_dir = os.path.join(raw_logs_folder, folder)

    # Read all logs as one
    log_files = [log for log in os.listdir(folder_server_dir) if log.endswith(extension_file_uncompessed)]
    log_files = sort_list_logs(log_files)

    if log_files:

        all_logs = ""
        for log_file in log_files:
            with open(os.path.join(folder_server_dir, log_file), 'r', encoding='utf-8') as file:
                log_contents = file.read().split('\n')
                # This wont work with 2 digits (get the date from the file name)
                log_contents = "\n".join([f'[{log_file[:-(2+len(extension_file_uncompessed))]}] ' + line for line in log_contents if line.strip() != ""])
                all_logs += log_contents + "\n"

        # Transform information it in meaningful information
        pattern = r'\[(\d{4}-\d{2}-\d{2})\] \[.*?(\d{2}:\d{2}:\d{2}).*?\] \[(.*?)/.*?\]: (.*?)\n'
        matches = re.findall(pattern, all_logs)

        # Create a list of dictionaries to store the extracted data
        log_data = [{'server_id': folder, 'date': match[0], 'time': match[1], 'category': match[2], 'information': match[3]} for match in matches]
        # Create a dataframe of the logs
        df_logs = pd.DataFrame(log_data)
        df_logs['user'] = None
        df_logs['activity'] = None

        # Filter by column category selecting only Server thread logs nad delete that column
        df_logs = df_logs[df_logs['category'] == 'Server thread'][['server_id', 'date', 'time', 'information', 'user', 'activity']]

        # Create a dataframe of the users
        df_users = pd.read_csv(os.path.join(folder_server_dir,'users.csv'))

        # Add information when server started
        index=df_logs[df_logs['information'].str.startswith("Starting minecraft server version")].index
        if not index.empty:
            df_logs.loc[index, 'user'] = 'server'
            df_logs.loc[index, 'activity'] = 'start'
        # Add information when server stopped
        index=df_logs[df_logs['information'].str.startswith("Stopping server")].index
        if not index.empty:
            df_logs.loc[index, 'user'] = 'server'
            df_logs.loc[index, 'activity'] = 'stop'
        # Add information when user did something
        for user in df_users['name']:
            # Add information when user activity
            index=df_logs[df_logs['information'].str.startswith(user)].index
            if not index.empty:
                df_logs.loc[index, 'user'] = user
                df_logs.loc[index, 'activity'] = 'action'
            # Add information when user login
            index=df_logs[df_logs['information'].str.startswith(f'{user} joined the game')].index
            if not index.empty:
                df_logs.loc[index, 'user'] = user
                df_logs.loc[index, 'activity'] = 'login'
            # Add information when user logout
            index=df_logs[df_logs['information'].str.startswith(f'{user} left the game')].index
            if not index.empty:
                df_logs.loc[index, 'user'] = user
                df_logs.loc[index, 'activity'] = 'logout'
            # Delete duplicated activity of login and logout
            index=df_logs[df_logs['information'].str.startswith(f'{user}[') | df_logs['information'].str.startswith(f'{user} lost connection: Disconnected')].index
            if not index.empty:
                df_logs.loc[index, 'user'] = None
                df_logs.loc[index, 'activity'] = None
        # Rows with no server/user activity is deleted
        df_logs = df_logs.dropna(subset=['activity'])

        # Save every log in df_all_logs for each iteration
        df_all_logs = pd.concat([df_all_logs, df_logs], ignore_index=True)

df_all_logs.to_parquet(os.path.join(output_logs_folder, 'activity.parquet'), index=False)

# Delete all logs inside each server folder
for folder in os.listdir(raw_logs_folder):
    folder_server_dir = os.path.join(raw_logs_folder, folder)
    [os.remove(os.path.join(folder_server_dir, log)) for log in os.listdir(folder_server_dir) if log.endswith(extension_file_uncompessed)]

## Load processed data into Data Warehouse (Postgres)

In [ ]:
# Read parquet file into a dataframe
all_new_activity=pd.read_parquet(os.path.join(output_logs_folder, 'activity.parquet'))

# Connect to database and upload all new logs into table
engine = create_engine(connection)
with engine.connect() as conn:

# Start a new transaction
    trans = conn.begin()

    try:
        # Load all new activity into postgres
        all_new_activity.to_sql(name = 'activity', schema = SCHEMA, con = conn, if_exists='append', index=False)
        # Commit the transaction
        trans.commit()

    except Exception as e:
        # Rollback the transaction on exception
        print('!!! [ERROR IN DATABASE QUERIES] !!!')
        trans.rollback()
        print('Transaction has been rolled back')
        print(f'Error occurred during transaction:\n{e}')
        raise